# The satellite-image-based surface reconstruction in `MicMac`, tuto #2

This tutorial is a continuation of the tutorial [01_basic_satellite.ipynb](https://github.com/micmacIGN/Documentation/blob/master/TutoJupyter/01_satellite_basic.ipynb). In contrast to the first tutorial, here, we will do multi-view surface reconstruction using epipolar images. After setting-up MicMac and downloading the dataset, the pipeline is as follows:
  1. Tie-points extraction
  2. RPC-bundle adjustement
  3. Automated pipeline for surface reconstruction from N images:

    3.1. ***Epipolar*** image resampling

    3.2. Re-creation of ***RPC localisation*** corresponding to the epipolar images 

    3.3. ***N stereo image matchings*** in image geometry

    3.4. ***Spatial similarity transformation*** of individual depth maps to a common frame

    3.5. ***Fusion***
  
<center>
  <img src="https://drive.google.com/uc?id=14riCQR0tz5Qa3JrOtcDQgOLvZOBgO-LJ" height=220pix/>
  <br> 
</center>
<center> Figure. General overview of the workflow.</center>




*Contact: ewelina.rupnik(at)ign.fr*

## Projet set-up

### Download and compile MicMac & dependencies

In [ ]:
import os
from os.path import exists, join, basename, splitext
import numpy as np 
import cv2
import matplotlib.pyplot as plt   

Dependencies_install = True
MicMac_clone = True
MicMac_cmake = True 
MicMac_build = True

YOUR_PATH = '/content/'#MyDrive/micmac/satellites/'
!cd $YOUR_PATH
!pwd


if Dependencies_install:
  !apt update
  !apt install -y cmake
  !pip install dlib
  !apt-get install imagemagick proj-bin exiv2
  !pip install wget gdown

if MicMac_clone:
  if not exists(YOUR_PATH+'micmac/'):
    git_repo_url = 'https://github.com/micmacIGN/micmac.git'
    !git clone $git_repo_url

if MicMac_cmake:
  !cd micmac
  if not exists(YOUR_PATH+'micmac/build'):
    !mkdir $YOUR_PATH"micmac/build"

  !cd $YOUR_PATH"micmac/build"
  !cmake $YOUR_PATH"micmac" -DBUILD_POISSON=OFF

  if MicMac_build:
    !make install -j28

### Add environmental variable & download the dataset

The dataset consists of: 
  * 4 images (tif)
  * 4 corresponding RPCs (xml)
  * `WGS84toUTM.xml` with the definition of a projection coordinate system (*proj4* format)

In [ ]:
import os
os.environ['PATH'] += ":/content/micmac/bin/"
!echo $PATH

# if you can see the commands printed to the screen, everything is OK
!mm3d

In [ ]:
# download data
dataset_url = 'https://drive.google.com/uc?id=18hmQL5kIqhcnR5ahp8IUsMZxLv7jvjgB' 
!gdown $dataset_url -O "satellite_data.tar.gz" 

# unpack
if not exists(YOUR_PATH+'satellite_data'):
  !mkdir $YOUR_PATH'satellite_data'
!tar -xf satellite_data.tar.gz -C $YOUR_PATH'satellite_data'
%cd $YOUR_PATH'satellite_data'

# utility functions to visualise tie-points
utils_url='https://drive.google.com/uc?id=1ATO1Nz_aXApxVnm6l7x1xappGXtcjuvp'
!gdown $utils_url -O "mm3d_utils.py"   

## 1. Extract SIFT tie-points

  - **computation strategy**: there exist several predefined *strategies* to compute tie-points: `Line`, `All`, `MulScale`, `File`. We will use the `All` strategy where tie-points are searched between all possible pairs. Refer to MicMac documentation for the other modes. 
  - **image resolution**: tie-points extraction is very costly, and to limit the computation time we usually downsampled the images; in this example, indicate resolution of `-1` which means full-resolution images; otherwise, if set to, e.g., `2000`, the images will be downsampled such that the larger image dimension (typically the width) will have `2000` pixels; the other dimension will have a size that is proportionally smaller;
  - **ExpTxt=1**: the extracted tie-points will be saved in a text format (as opossed to the default dat format)
  - **results**: tie-points are stored in the `Homol` directory. For instance, tie-points correponding to image `Im1.tif` will be stored in `Homol/PastisIm1.tif/` directory. If `Im1.tif` overlaps with `Im2.tif` and `Im3.tif`, their tie-points will be stored in `Homol/PastisIm1.tif/Im2.tif.dat` and `Homol/PastisIm1.tif/Im3.tif.dat`, respectively. If you chose to export in the text format, the `dat` extension will be replaced with `txt`.


*Note: Intermediary results are stored in the `Pastis` directory. It takesa  significant amount of space and is not used at later processing stages, therefore you may delete it.*

In [ ]:
!mm3d Tapioca All .*tif -1 ExpTxt=1 @ExitOnBrkp

### Visualise tie-points

Read any pair of images and visalise their tie-points.

In [ ]:
import mm3d_utils 

aIm1 = cv2.imread('TPMM_0435.tif',cv2.IMREAD_IGNORE_ORIENTATION)
aIm2 = cv2.imread('TPMM_0566.tif',cv2.IMREAD_IGNORE_ORIENTATION) 
 
TPtsVec = mm3d_utils.ImportHom("Homol/PastisTPMM_0435.tif/TPMM_0566.tif.txt") 
 
mm3d_utils.plot_images([np.asarray(aIm1),np.asarray(aIm2)]) 
mm3d_utils.plot_tiepts2(np.asarray(TPtsVec,dtype=float))

## 2. RPC-bundle adjustment

### Read the RPCs in DIMAP format

This function reads the DIMAP format RPCs and converts it to a *MicMac* format. Several parameters are specified here:

  - `(.*).tif` this is the pattern of input images (note the dot preceding the star which is the posix convention)

  - `\$1.xml` is the corresponding pattern of RPC files; I use here a regular expression that associates the image name with its corresponding RPC file name; you may also run the command independently for each image if you're not familiar with regular expressions;

  - `RPC-d0` is the directory name where the converted files will be stored; it will serve as input in the following step, i.e., the bundle adjustment;

  - `Degre=0`, the degree of the polynomial correction;

  By choosing a zero-degree polynomial we will correct the satellite's geolocalisation by modelling a 3D image shift; please refer to [Rupnik et al., 2016] for more on the method.

  - `ChSys=WGS84toUTM.xml` definition of the projection coordinate sytem; MicMac expects that the processing coordinate frame is euclidean and all three coordinates have the same unit. The RPCs are expressed in geographical coordinates which are neither euclidean, nor unique in terms of units. To overcome that, MicMac will transfer, on the fly, the RPCs to a user-defined coordinate system, in this exemple defined in the `WGS84toUTM.xml` file. The definition of the coordinate system follows the `proj4` library convention. You can retrieve the code corresponding to the coordinate frame of your interest from `https://spatialreference.org/`



<sub> Rupnik, E., Deseilligny, M.P., Delorme, A. and Klinger, Y., 2016. Refined satellite image orientation in the free open-source photogrammetric tools Apero/Micmac. ISPRS Annals of the Photogrammetry, Remote Sensing and Spatial Information Sciences, 3, p.83.<sub>

In [ ]:
!mm3d Convert2GenBundle -help @ExitOnBrkp
!mm3d Convert2GenBundle "(.*).tif" "\$1.xml" RPC-d0 ChSys=WGS84toUTM.xml Degre=0  @ExitOnBrkp

### Run the adjustment

Within the bundle adjustemnt, MicMac will estimate the parameters of $D_x$ and $D_y$ functions, that are the polynomials you have defined in `Convert2GenBundle` method. In this example our observations are the tie-points. It is also possible to include ground control points.


$\begin{equation}
\begin{split}
y = g (\phi, \lambda, h) +  {D_y (x,y)}\\
x = h (\phi, \lambda, h) +  {D_x (x,y)}
\end{split}
\end{equation}$

where $g$ and $h$ are the RPC functions, and 

$\begin{equation}
\begin{split}
D_y (x,y) = \sum_{i=0}^m \sum_{j=0}^n a_{ij} \cdot x^i y^j \\
D_x (x,y) = \sum_{i=0}^m \sum_{j=0}^n b_{ij} \cdot x^i y^j
\end{split}
\end{equation}$


The input parameters:
- `RPC-d0` is the folder with the initial geolocalisation
- `RPC-d0_adj` is the folder where the adjusted geoloc is saved
- `ExpTxt=1` indicates that tie-points are stored in text format 

In [ ]:
!mm3d Campari ".*tif" RPC-d0 RPC-d0-adj ExpTxt=1 @ExitOnBrkp

### Interpreting the results

One way to asses the quality of the adjustment is to look at the tie-points residuals (for more sophisticated quality estimates see `MMTestOrient` in MicMac documentation). 

The bundle adjustment is carried out in several iterations. Let's look at image `TPMM_0435.tif` in the last iteration:

> `RES:[TPMM_0435.tif][g] ER2 0.24636 Nn 100 Of 11753 Mul 5171 Mul-NN 5171 Time 1.15821`

* `0.24636` pixels is the mean residual calculated over all tie-points (i.e., $\sigma$ of the bundle)  

* `Nn 100` means that 100$\%$ of tie-points were considered as inliers

* `11753` there were as many tie-points found

* `5171` there were as many multiple tie-points found (out of the `11753`), i.e., tie-points observed in at least 3 images;

## Automated pipeline for surface reconstruction

### Compute the surface

<center>
  <img src="https://drive.google.com/uc?id=1LgLh9rw4yW3VZqKiIFnBp8B8SszODuQq" height=220pix/>
  <br> 
</center>
<center> Figure. SAT4Geo pipeline.</center>
<br>

The pipeline runs with `mm3d SAT4Geo` and consists of several sub-modules:
  - `SAT4Geo_Pairs` defines image pairs for which the image matching will be later computed; the result is stored in *Pairs.xml*; there are two $criteria$ applied when selecting pairs:  
      -  $\frac{B}{H}$ range is set by default to   $\in \left< 0.01, 0.3 \right>$; this range can be modified with the `BH` parameter;
      - images must overlap.

    You may as well skip that step and provide the pipeline with your own pairs. To do that, put your pairs in the *xml* format file MicMac likes, and use the `Pairs` parameter, for instance `Pairs=YourOwnPairs.xml`
      

  - `SAT4Geo_CreateEpip` creates epipolar images for a set of pairs defined in  *Pairs.xml*; it implements the approach presented in [1]

  - `SAT4Geo_EpiRPC` calculates RPC functions for the images in their new geometry (i.e., epipolar geometry); by default the new RPCs are stored in `Ori-EpiRPC`

  - `SAT4Geo_MM1P` carries out image matching for all image pairs in *Pairs.xml*; if your epipolar images were created outside MicMac, put their names inside the *xml* and MicMac shall know how to proceed; however, note that unless you provide the RPC files corresponding to your pairs, MicMac will not be able to perform the latter fusion;
      * the per-pair depth maps are found in seperate directories following this naming convention - `MEC-Cple_X_Y/`; their respective names are also stored in  `PairsDirMEC.xml`;


  - `SAT4Geo_Fuse` fuses the per-epipolar-pair depth maps into a single and (hopefully) complete surface; it is the xact same algorithm that was described in  [01_basic_satellite.ipynb](https://github.com/micmacIGN/Documentation/blob/master/TutoJupyter/01_satellite_basic.ipynb)

    * The result is saved to `Fusion/Fusion_Prof.tif`, there is a corresponding mask and a correlation map named with `_Mask` and `Correl` postfixes, respectively.



If you're interested in tweaking default parameters' values, type `-help`. In this `mm3d SAT4Geo` example, the parameters are:

* `.*tif` is the pattern of input images
* `RPC-d0-adj` is the name of the directory with the RPCs (prefixed by `Ori-`)
* `ChSys=WGS84toUTM.xml` indicates the file that defines the processing coordinate frame
* `NbP=24` is the number of parallel processes (make sure it's less than the number of cores your PC has)



<sub> [1] Marc Pierrot Deseilligny \& Ewelina Rupnik. Epipolar rectification of a generic camera. 2020. ⟨hal-02968078⟩ 

 

In [ ]:
# the Exe=0 means "no execution" but display commands that would be otherwise launched
!mm3d SAT4Geo ".*tif" RPC-d0-adj ChSys=WGS84toUTM.xml NbP=24 Exe=0

In [ ]:
!mm3d SAT4Geo ".*tif" RPC-d0-adj ChSys=WGS84toUTM.xml NbP=24 Exe=1

### Create a grayshaded DSM

Represent the surface in form of a grayshading. To visually asses the quality of your surface, it is much more intuitive than just looking at the depth/Z image. 

In [ ]:
!mm3d GrShade Fusion/Fusion_Prof.tif ModeOmbre=IgnE Mask=Fusion/Fusion_Masq.tif @ExitOnBrkp

### Visualise the grayshaded surface

In [ ]:
surface_shade_im = cv2.imread("Fusion/Fusion_ProfShade.tif",cv2.IMREAD_IGNORE_ORIENTATION)

fig, ax = plt.subplots(figsize=(30, 10))
ax.imshow(surface_shade_im,cmap="gray") 
plt.tight_layout()